In [1]:
#import the libraries as needed
from selenium import webdriver
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm # to create loadbard in for loop
import os
from datetime import date
import time
import random
import pickle

For now dowload all earning call transcript for a specified ticker

### url to use for apple
https://seekingalpha.com/api/v3/symbols/aapl/transcripts?filter[until]=undefined&id=aapl&include=author%2CprimaryTickers%2CsecondaryTickers%2Csentiments&isMounting=true&page[size]=20

https://seekingalpha.com/api/v3/symbols/aapl/transcripts?filter[until]=1501632962&id=aapl&include=author%2CprimaryTickers%2CsecondaryTickers%2Csentiments&isMounting=false&page[size]=20

In [2]:
def create_url_for_scrap(ticker, publish_time_min ):
    # accept publis_time_min = None
    return_per_request  = 20 #the one used by seekingalpha by default
    ticker = ticker.lower()
    publish_time_min = "undefined" if publish_time_min ==None else str(publish_time_min)
    to_return = ("https://seekingalpha.com/api/v3/symbols/"+ticker+"/transcripts?filter[until]="+publish_time_min
                 +"&id="+ticker
                 +"&include=author%2CprimaryTickers%2CsecondaryTickers%2Csentiments&isMounting=true&page[size]="
                 + str(return_per_request))
    return to_return
def is_earning_call(element_response):
    title = element_response["attributes"]["title"].lower()
    has_earning_in_title = "earning" in title or "earnings" in title
    has_call_in_title = "call" in title
    return element_response["type"]=="transcript" and has_call_in_title and has_earning_in_title
def get_url_id_date_earning_call(element_response):
    date = element_response["attributes"]["publishOn"][:10]
    call_id = element_response["id"]
    url = element_response["links"]["self"]
    return url, call_id, date

In [6]:
# List of all S&P 500 tickers
import pandas as pd
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df_tickers = table[0]
df_tickers.to_csv('S&P500-Info.csv')
df_tickers.to_csv("S&P500-Symbols.csv", columns=['Symbol'])
tickers=df_tickers["Symbol"].values
print(tickers)

['MMM' 'AOS' 'ABT' 'ABBV' 'ABMD' 'ACN' 'ATVI' 'ADM' 'ADBE' 'ADP' 'AAP'
 'AES' 'AFL' 'A' 'AIG' 'APD' 'AKAM' 'ALK' 'ALB' 'ARE' 'ALGN' 'ALLE' 'LNT'
 'ALL' 'GOOGL' 'GOOG' 'MO' 'AMZN' 'AMCR' 'AMD' 'AEE' 'AAL' 'AEP' 'AXP'
 'AMT' 'AWK' 'AMP' 'ABC' 'AME' 'AMGN' 'APH' 'ADI' 'ANSS' 'ANTM' 'AON'
 'APA' 'AAPL' 'AMAT' 'APTV' 'ANET' 'AIZ' 'T' 'ATO' 'ADSK' 'AZO' 'AVB'
 'AVY' 'BKR' 'BALL' 'BAC' 'BBWI' 'BAX' 'BDX' 'WRB' 'BRK.B' 'BBY' 'BIO'
 'TECH' 'BIIB' 'BLK' 'BK' 'BA' 'BKNG' 'BWA' 'BXP' 'BSX' 'BMY' 'AVGO' 'BR'
 'BRO' 'BF.B' 'CHRW' 'CDNS' 'CZR' 'CPT' 'CPB' 'COF' 'CAH' 'KMX' 'CCL'
 'CARR' 'CTLT' 'CAT' 'CBOE' 'CBRE' 'CDW' 'CE' 'CNC' 'CNP' 'CDAY' 'CERN'
 'CF' 'CRL' 'SCHW' 'CHTR' 'CVX' 'CMG' 'CB' 'CHD' 'CI' 'CINF' 'CTAS' 'CSCO'
 'C' 'CFG' 'CTXS' 'CLX' 'CME' 'CMS' 'KO' 'CTSH' 'CL' 'CMCSA' 'CMA' 'CAG'
 'COP' 'ED' 'STZ' 'CEG' 'COO' 'CPRT' 'GLW' 'CTVA' 'COST' 'CTRA' 'CCI'
 'CSX' 'CMI' 'CVS' 'DHI' 'DHR' 'DRI' 'DVA' 'DE' 'DAL' 'XRAY' 'DVN' 'DXCM'
 'FANG' 'DLR' 'DFS' 'DISH' 'DIS' 'DG' 'DLTR' 'D' 'DPZ' 'DOV' 'DOW

In [12]:
# for now scrapp all earning call transcript from one company
for ticker in ['ABT']:
#for ticker in tickers:
#for ticker in ['MMM','AOS','ABT']:
    html_path = "data/ticker/"+ticker ## todo save file with name time step*
    last_time_scrap_path =html_path+ "/last_time_scrap"
    if not os.path.exists(html_path):
        os.makedirs(html_path)
        print(f"The new directory {html_path} has been created!")
    if os.path.exists(last_time_scrap_path):
        with open(last_time_scrap_path, "r") as file:
            publish_time_min = file.read()
    else : 
        publish_time_min = None
    if publish_time_min !="all":
        more_data = True
        executable_path = "./chromedriver"
        service = Service(executable_path)
        option = webdriver.ChromeOptions()
        driver = webdriver.Chrome(service=service,options=option)
        id_URLs = []
        id_to_date = {}
        while more_data:
            url = create_url_for_scrap(ticker, publish_time_min)
            driver.get(url)
            json_response = driver.find_element(by=By.TAG_NAME, value = "body").text
            data_responses = json.loads(json_response)
            if "meta" not in data_responses:
                time.sleep(5) # 2 chances to be flag as robot
                driver.get(url)
                json_response = driver.find_element(by=By.TAG_NAME, value = "body").text
                data_responses = json.loads(json_response)
                if "meta" not in data_responses:
                    last_publish_time_min = publish_time_min
                    break
            publish_time_min = str(data_responses["meta"]["page"]["minmaxPublishOn"]["min"]) # need to be string
            if publish_time_min!= 'None':
                for element in data_responses["data"]:
                    if is_earning_call(element):
                        URL, call_id, date = get_url_id_date_earning_call(element)
                        id_URLs.append((call_id,URL))
                        id_to_date[call_id] = date
            else :
                last_publish_time_min = "all"
                more_data = False
            time.sleep(2+random.uniform(0, 1))
            
    # Download HTML files
    domain_name = 'https://seekingalpha.com'
    for call_id, URL in tqdm(id_URLs):
        file_path = html_path + "/"+ call_id + ".html"
        with open(file_path, "w") as file :
            url = domain_name+URL
            html =  requests.get(url).text
            file.write(html)
            
    # Save the last scrapped url of the ticker, so that if bot detected can resume
    with open(html_path+"/last_time_scrap", "w") as file:
        file.write(last_publish_time_min)    
    
    # Check last file got downloaded, i.e. captcha/bot error
    #BeautifulSoup(html).get_text()[:2000]
    
    # Save date of transcripts
    id_to_date_path = "data/ticker/id_to_date"
    if not os.path.exists(id_to_date_path):
        file_id_to_date = {}
    else:
        with open(id_to_date_path,"rb") as file :
            file_id_to_date = pickle.load(file)
    for call_id, date in id_to_date.items():
        file_id_to_date[call_id] = date

    with open(id_to_date_path,"wb") as file : 
        pickle.dump(file_id_to_date, file, protocol=pickle.HIGHEST_PROTOCOL)


ABT


0it [00:00, ?it/s]

Download HTML files 

domain_name = 'https://seekingalpha.com'
for call_id, URL in tqdm(id_URLs):
    file_path = html_path + "/"+ call_id + ".html"
    with open(file_path, "w") as file :
        url = domain_name+URL
        html =  requests.get(url).text
        file.write(html)

Save the last scrapped url of the ticker, so that if bot detected can resume

with open(html_path+"/last_time_scrap", "w") as file:
    file.write(last_publish_time_min)

## Check last file got downloaded, i.e. captcha/bot error 

BeautifulSoup(html).get_text()[:2000]

Save date of transcripts

id_to_date_path = "data/ticker/id_to_date"
if not os.path.exists(id_to_date_path):
    file_id_to_date = {}
else:
    with open(id_to_date_path,"rb") as file :
        file_id_to_date = pickle.load(file)
for call_id, date in id_to_date.items():
    file_id_to_date[call_id] = date
    
with open(id_to_date_path,"wb") as file : 
    pickle.dump(file_id_to_date, file, protocol=pickle.HIGHEST_PROTOCOL)
    